In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(300, 300, 3),
                                               include_top=False,
                                               weights='imagenet')

C:\Users\AERO\AppData\Local\Temp\ipykernel_12648\1061583300.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(300, 300, 3),


In [3]:
def preprocess_dataset(img, label):
    img = tf.divide(tf.cast(img, tf.float32), 255.)
    #encoded_label = tf.one_hot(label, depth=10)
    return img, label

In [4]:
ds, info = tfds.load("horses_or_humans", split=["train[:80%]", "train[80%:]", "test"], as_supervised=True, with_info=True)
ds_train = ds[0]
ds_val = ds[1]
ds_test = ds[2]
ds_train = ds_train.map(preprocess_dataset).shuffle(1000).batch(64)
ds_val = ds_val.map(preprocess_dataset).batch(64)
ds_test = ds_test.map(preprocess_dataset).batch(64)

In [10]:
image_batch, label_batch = next(iter(ds_train))
feature_batch = base_model(image_batch)
feature_batch.shape

TensorShape([64, 10, 10, 1280])

In [12]:
type(feature_batch)

tensorflow.python.framework.ops.EagerTensor

In [14]:
type(base_model)

keras.src.models.functional.Functional

In [16]:
base_model.summary()

Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 150, 150,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 150, 150,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 150, 150,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 150, 150,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 150, 150,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 150, 150,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 150, 150,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 150, 150,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 151, 151,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 75, 75,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 75, 75,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 75, 75,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 75, 75,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

## freaze

In [19]:
base_model.trainable = False

In [21]:
base_model.summary()

Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 150, 150,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 150, 150,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 150, 150,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 150, 150,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 150, 150,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 150, 150,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 150, 150,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 150, 150,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 150, 150,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 151, 151,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 75, 75,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 75, 75,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 75, 75,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 75, 75,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

## Add a classification head

In [30]:
Global_average_pool =tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = Global_average_pool(feature_batch)
print(feature_batch_average.shape)

(64, 1280)


In [35]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(64, 1)


## create model

In [48]:
inputs = tf.keras.layers.Input(shape= (300, 300, 3))
x = base_model(inputs =inputs , training = False )
x = Global_average_pool(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs = inputs , outputs = outputs)

In [53]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 10, 10, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Compile the model

In [63]:
rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=rate),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [66]:
history = model.fit(ds_train,
                    epochs=10,
                    validation_data=ds_val)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 921ms/step - accuracy: 0.4515 - loss: 1.0184 - val_accuracy: 0.5659 - val_loss: 0.8279
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 854ms/step - accuracy: 0.4421 - loss: 0.9048 - val_accuracy: 0.5610 - val_loss: 0.7619
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 930ms/step - accuracy: 0.4544 - loss: 0.8186 - val_accuracy: 0.5561 - val_loss: 0.7042
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 906ms/step - accuracy: 0.4793 - loss: 0.7341 - val_accuracy: 0.5707 - val_loss: 0.6516
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 905ms/step - accuracy: 0.5253 - loss: 0.6470 - val_accuracy: 0.6439 - val_loss: 0.6018
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 929ms/step - accuracy: 0.5595 - loss: 0.6095 - val_accuracy: 0.6634 - val_loss: 0.5546
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 881ms/step - accuracy: 0.6306 - loss: 0.5605 - val_accuracy: 0.7122 - val_loss: 0.5125
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 907ms/step - accuracy: 0.7066 - loss: 0.5048 - val_accu

برای مدل واقعی که ما معمولا انقدر خوب جواب نمیگیریم که برای همین نیازه که چندلایه را از حالت فریز خارج کنیم و بذاریم مدله یاد بگیره پس از روش زیر استفاده می کنیم

از لایه های آخر دونه دونه از فریز درمیاریم لایه هارو 

## fine tuning

In [82]:
base_model.trainable = True

print('number of base model layers :',len(base_model.layers))

number of base model layers : 154


In [86]:
fine_tuning = 100
for layer in base_model.layers[:fine_tuning] : 
    layer.trainable = False

now compile

In [93]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=rate/10),
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 10, 10, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,862,721 (7.11 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [95]:
history.epoch

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [97]:
history_fine = model.fit(ds_train,
                         epochs=20,
                         # چون در ادامه ی مدل قبلی می خواهیم ترین شود و نه از اول می ذاریم از آخرین لایه ی آموزشی ما یاد بگیرد
                         #در پایین هم می بینم که از 10 شروع می شود
                         initial_epoch=history.epoch[-1],
                         validation_data=ds_val)

Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.8621 - loss: 0.3841 - val_accuracy: 0.9951 - val_loss: 0.2300
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9931 - loss: 0.2032 - val_accuracy: 0.9951 - val_loss: 0.1607
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 1.0000 - loss: 0.1238 - val_accuracy: 0.9951 - val_loss: 0.1146
Epoch 13/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 1.0000 - loss: 0.0793 - val_accuracy: 0.9951 - val_loss: 0.0836
Epoch 14/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9991 - loss: 0.0593 - val_accuracy: 0.9951 - val_loss: 0.0614
Epoch 15/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 1.0000 - loss: 0.0360 - val_accuracy: 0.9951 - val_loss: 0.0459
Epoch 16/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 1.0000 - loss: 0.0246 - val_accuracy: 0.9951 - val_loss: 0.0351
Epoch 17/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 1.0000 - loss: 0.0169 - val_accuracy: 1.0000 - v

In [100]:
model.evaluate(ds_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 680ms/step - accuracy: 0.9818 - loss: 0.0722


[0.08004135638475418, 0.98046875]